# Making API Calls
- 1 call per minute for every hour from today till 2018
- Final Dataset contain columns -- carpark_number	update_datetime	total_C	total_H	total_Y	avail_C	avail_H	avail_Y

In [ ]:
from datetime import datetime
from datetime import timedelta
import requests
import pandas as pd
import json
import time

def flatten_data(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out
    
start = datetime.strptime('2018-01-01/00:01:00', '%Y-%m-%d/%H:%M:%S')
end = datetime.now()
skipped = []

data = requests.get('https://api.data.gov.sg/v1/transport/carpark-availability?date_time={}'.format('T'.join(str(start).split()))) 
df_list = []
j=0
for y in json.loads(data.text)['items'][0]['carpark_data']:
    df_list.append(flatten_data(y))
df = pd.DataFrame(df_list)

while start<end:
    try:
        data = requests.get('https://api.data.gov.sg/v1/transport/carpark-availability?date_time={}'.format('T'.join(str(start).split()))) 
        df_list = []
        for y in json.loads(data.text)['items'][0]['carpark_data']:
            df_list.append(flatten_data(y))

        if len(df)==0:
            df = pd.DataFrame(df_list)
        else:
            df1 = pd.DataFrame(df_list)
            df = pd.concat((df,df1))
        j+=1
        if j%100 == 0:
            df.drop_duplicates().to_csv('data{}.csv'.format(j//100),index=False)
            df = df.iloc[0:0]
        time.sleep(0.3)
        start = start + timedelta(hours=1)
        print(start)
    except Exception as e: 
        print(e)
        skipped.append(start)
        start = start + timedelta(hours=1)
with open('skipped.txt','a') as file:
    file.write(str(skipped))

In [ ]:
import pandas as pd

i=2
df = pd.read_csv('data1.csv'.format(i))
loop = True
while True: ##EDIT THIS 
    try:
        df = pd.concat((df,pd.read_csv('data{}.csv'.format(i))))
        i+=1
    except:
        break

df = df.drop_duplicates()

df['total_C'] = 0
df.loc[df.carpark_info_0_lot_type=='C','total_C'] = df.loc[df.carpark_info_0_lot_type=='C']['carpark_info_0_total_lots']
df.loc[df.carpark_info_1_lot_type=='C','total_C'] = df.loc[df.carpark_info_1_lot_type=='C']['carpark_info_1_total_lots']
df.loc[df.carpark_info_2_lot_type=='C','total_C'] = df.loc[df.carpark_info_2_lot_type=='C']['carpark_info_2_total_lots']

df['total_H'] = 0
df.loc[df.carpark_info_0_lot_type=='H','total_H'] = df.loc[df.carpark_info_0_lot_type=='H']['carpark_info_0_total_lots']
df.loc[df.carpark_info_1_lot_type=='H','total_H'] = df.loc[df.carpark_info_1_lot_type=='H']['carpark_info_1_total_lots']
df.loc[df.carpark_info_2_lot_type=='H','total_H'] = df.loc[df.carpark_info_2_lot_type=='H']['carpark_info_2_total_lots']

df['total_Y'] = 0
df.loc[df.carpark_info_0_lot_type=='Y','total_Y'] = df.loc[df.carpark_info_0_lot_type=='Y']['carpark_info_0_total_lots']
df.loc[df.carpark_info_1_lot_type=='Y','total_Y'] = df.loc[df.carpark_info_1_lot_type=='Y']['carpark_info_1_total_lots']
df.loc[df.carpark_info_2_lot_type=='Y','total_Y'] = df.loc[df.carpark_info_2_lot_type=='Y']['carpark_info_2_total_lots']

df['avail_C'] = 0
df.loc[df.carpark_info_0_lot_type=='C','avail_C'] = df.loc[df.carpark_info_0_lot_type=='C']['carpark_info_0_lots_available']
df.loc[df.carpark_info_1_lot_type=='C','avail_C'] = df.loc[df.carpark_info_1_lot_type=='C']['carpark_info_1_lots_available']
df.loc[df.carpark_info_2_lot_type=='C','avail_C'] = df.loc[df.carpark_info_2_lot_type=='C']['carpark_info_2_lots_available']

df['avail_H'] = 0
df.loc[df.carpark_info_0_lot_type=='H','avail_H'] = df.loc[df.carpark_info_0_lot_type=='H']['carpark_info_0_lots_available']
df.loc[df.carpark_info_1_lot_type=='H','avail_H'] = df.loc[df.carpark_info_1_lot_type=='H']['carpark_info_1_lots_available']
df.loc[df.carpark_info_2_lot_type=='H','avail_H'] = df.loc[df.carpark_info_2_lot_type=='H']['carpark_info_2_lots_available']

df['avail_Y'] = 0
df.loc[df.carpark_info_0_lot_type=='Y','avail_Y'] = df.loc[df.carpark_info_0_lot_type=='Y']['carpark_info_0_lots_available']
df.loc[df.carpark_info_1_lot_type=='Y','avail_Y'] = df.loc[df.carpark_info_1_lot_type=='Y']['carpark_info_1_lots_available']
df.loc[df.carpark_info_2_lot_type=='Y','avail_Y'] = df.loc[df.carpark_info_2_lot_type=='Y']['carpark_info_2_lots_available']

df = df.drop(labels = ['carpark_info_0_total_lots','carpark_info_0_lot_type','carpark_info_0_lots_available','carpark_info_1_total_lots','carpark_info_1_lot_type','carpark_info_1_lots_available','carpark_info_2_total_lots','carpark_info_2_lot_type','carpark_info_2_lots_available'],axis=1)
df

,carpark_number,update_datetime,total_C,total_H,total_Y,avail_C,avail_H,avail_Y
0,HE12,2021-12-31T23:38:31,105,0,0,57,0,0
1,HLM,2021-12-31T23:38:25,583,0,0,485,0,0
2,RHM,2021-12-31T23:38:31,329,0,0,142,0,0
3,BM29,2021-12-31T23:38:26,97,0,0,83,0,0
4,Q81,2021-12-31T23:38:37,96,0,0,75,0,0
...,...,...,...,...,...,...,...,...
177754,JB2,2022-09-09T15:43:42,0,0,21,0,0,12
177755,BE3,2022-09-09T15:58:56,306,0,0,162,0,0
177756,BE3R,2022-09-09T15:58:57,195,0,0,100,0,0
177757,B8B,2022-09-09T15:58:22,249,0,0,162,0,0
